# Import Libraries

In [4]:
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
from lime import lime_image
from skimage.segmentation import mark_boundaries

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Clear session
tf.keras.backend.clear_session()

# Set parameters
IMG_SIZE = (128, 128)  # Increased image size
BATCH_SIZE = 16  # Increased batch size
base_dir = 'E:/val'
categories = ['covid-19', 'non-covid', 'normal']


# Data Loading and Preprocessing


In [8]:
# Function to load and preprocess data
def load_and_preprocess_data(base_dir, categories, img_size):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(base_dir, category, 'images')
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path)
                img_array = cv2.resize(img_array, img_size)
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                img_array = cv2.GaussianBlur(img_array, (5, 5), 0)
                img_array = cv2.equalizeHist(img_array)
                img_array = img_array / 255.0
                img_array = np.expand_dims(img_array, axis=-1)
                img_array = np.repeat(img_array, 3, axis=-1)
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                print(f"Error loading image {img}: {e}")
    return np.array(data), np.array(labels)

# Load and preprocess data
data, labels = load_and_preprocess_data(base_dir, categories, IMG_SIZE)
labels = to_categorical(labels, num_classes=len(categories))

# Check for NaN values in the data
if np.isnan(data).any() or np.isnan(labels).any():
    print("NaN values found in the data or labels. Please clean the data.")
    data = np.nan_to_num(data)
    labels = np.nan_to_num(labels)


# Data Augmentation


In [9]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Function to plot training history
def plot_history(history, title):
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(title)
    plt.legend(loc='lower right')
    plt.show()

# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
